In [1]:
import joblib
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

import configuration.configuration as configuration
from mel_spectrogram.mel_spectrogram import MelSpectrogramGenerator
from notebook_utils import notebookToPython
from readers.label_reader import readTrainingLabelsWithJob

In [2]:
config = configuration.ConfigLoader('config.yml')

notebookToPython(config.projectName)
job = config.getJobConfig(config.activeJobId)

Write python file


In [3]:
generator = MelSpectrogramGenerator()
X, y_encoded = generator.generateMelSpectrograms(job, job.trainingDataPathSuffix)

Loading C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt...
fullDataPath: C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_train/flac
Loading audio files: 1269
Loading audio files: 2538
Loading audio files: 3807
Loading audio files: 5076
Loading audio files: 6345
Loading audio files: 7614
Loading audio files: 8883
Loading audio files: 10152
Loading audio files: 11421
Loading audio files: 12690
Loading audio files: 13959
Loading audio files: 15228
Loading audio files: 16497
Loading audio files: 17766
Loading audio files: 19035
Loading audio files: 20304
Loading audio files: 21573
Loading audio files: 22842
Loading audio files: 24111
Loading audio files: 25380
Number of audio files load: 25380


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2)    # test data is 20% of all data

In [5]:
import model_definitions.model_cnn_definition as model_cnn_definition

modelDef = model_cnn_definition.ModelCnnDefinition(job, X_train.shape[2], 1)
model = modelDef.buildModel()

__job__: <configuration.configuration.Job object at 0x000002DFD259B380>


In [6]:
model.compile(optimizer=job.optimizer, loss=job.loss, metrics=job.metrics)

In [7]:
# Train the Model
model.fit(X_train, y_train, batch_size=job.batchSize, epochs=job.numEpochs, validation_data=(X_test, y_test))

Epoch 1/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 39s 60ms/step - accuracy: 0.8843 - loss: 6.2471 - val_accuracy: 0.9003 - val_loss: 0.2382
Epoch 2/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 40s 62ms/step - accuracy: 0.9060 - loss: 0.2367 - val_accuracy: 0.9086 - val_loss: 0.2301
Epoch 3/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 38s 60ms/step - accuracy: 0.9141 - loss: 0.2096 - val_accuracy: 0.9194 - val_loss: 0.1982
Epoch 4/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 39s 62ms/step - accuracy: 0.9277 - loss: 0.1819 - val_accuracy: 0.9287 - val_loss: 0.1746
Epoch 5/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 40s 62ms/step - accuracy: 0.9379 - loss: 0.1594 - val_accuracy: 0.9634 - val_loss: 0.1065
Epoch 6/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 40s 62ms/step - accuracy: 0.9536 - loss: 0.1251 - val_accuracy: 0.9740 - val_loss: 0.0752
Epoch 7/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 40s 62ms/step - accuracy: 0.9711 - loss: 0.0821 - val_accuracy: 0.9750 - val_loss: 0.0684
Epoch 8/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 40s 64ms/step - accuracy: 0.9764 - loss: 0.0654 - 

In [8]:
joblib.dump(model, job.persistedModel)

['ASVspoof-2019-1_2025-03-16T18-16-03.333177.libjob']

### Test Model

In [9]:
import joblib
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

import configuration.configuration as configuration
from notebook_utils import notebookToPython
from readers.label_reader import readLabelsWithJob

In [10]:
oldJob = job
config = configuration.ConfigLoader('config.yml')

notebookToPython(config.projectName)
job = config.getJobConfig(config.activeJobId)

if (len(oldJob.persistedModel) > 0 and oldJob.persistedModel != job.persistedModel):
    job.persistedModel = oldJob.persistedModel


Write python file


In [11]:
model = joblib.load(job.persistedModel)

In [12]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
y_pred

159/159 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step


array([0, 0, 0, ..., 0, 0, 0])

In [13]:
y_test

array([0, 0, 0, ..., 0, 0, 0])

In [14]:
import json
from datetime import datetime
import pytz

score = accuracy_score(y_test, y_pred)

timestamp_utc = datetime.now(pytz.utc)

with open(job.persistedModelResults, "w") as file:
    file.write(f"job completed: {timestamp_utc.isoformat()}\n")
    file.write(f"model file: {job.persistedModel}\n")
    file.write(f"accuracy_score: {score}\n\n")
    prettyJson = json.dumps(job.__dict__, indent=4)
    file.write(f"job: {prettyJson}\n")